In [1]:
import pandas as pd

In [3]:
df_sec = pd.read_parquet("../app/portprop_data/security_info.parquet")
df_sec.head()

,sec_id,symbol,port_prop_id,port_type_id,currency_code,geography_id,top_pick_flag,hedging_cost_flag,owner,allow_manual_add,expected_return,expected_return_overwrite,portprop_class_overwrite,created_date,created_by,updated_date,updated_by
0,78309,AAAAAAAAZZZ,53,4,THB,2,False,None,superpat,True,5.00,False,False,NaT,None,NaT,None
1,78310,,29,4,THB,2,False,None,superpat,True,5.00,False,False,NaT,None,NaT,None
2,2,BOT25NA,18,3,THB,2,False,False,None,True,0.75,False,False,2024-07-15 17:38:30.950,jangssit,2024-07-15 19:09:42.467,goanywhere
3,3,BOTS149A,18,3,THB,2,False,False,None,True,0.75,False,False,2024-07-15 17:38:30.950,jangssit,2024-07-15 19:09:42.467,goanywhere
4,4,BP125A,18,3,THB,2,False,False,None,True,0.75,False,False,2024-07-15 17:38:30.950,jangssit,2024-07-15 19:09:42.467,goanywhere


In [9]:
df_pp = pd.read_parquet('../app/portprop_data/port_prop_class.parquet')
df_asset_class = pd.read_parquet('../app/portprop_data/ref_asset_class.parquet')
df_asset_sub_class = pd.read_parquet('../app/portprop_data/ref_portprop_asset_sub_class.parquet')

In [ ]:
df_sec_all = (df_sec[[
    'sec_id', 'symbol', 'port_prop_id', 'port_type_id', 'currency_code', 'geography_id',
    'top_pick_flag', 'hedging_cost_flag','owner', 'allow_manual_add',
    'expected_return', 'expected_return_overwrite', 'portprop_class_overwrite'
    ]].
    merge(df_pp[['port_prop_id', 'asset_class_id', 'portprop_sub_class_id', 'bm_id', 'expected_return_rate']], on='port_prop_id', how='left') \
    .merge(df_asset_class[['asset_class_id', 'asset_class_code', 'asset_class_name']], on='asset_class_id', how='left') \
    .merge(df_asset_sub_class[['portprop_sub_class_id', 'pp_asset_sub_class', 'pp_asset_sub_class_code']], on='portprop_sub_class_id', how='left')
)

In [18]:
df_sec_all.head().T

,0,1,2,3,4
sec_id,78309,78310,2,3,4
symbol,AAAAAAAAZZZ,,BOT25NA,BOTS149A,BP125A
port_prop_id,53,29,18,18,18
port_type_id,4,4,3,3,3
currency_code,THB,THB,THB,THB,THB
geography_id,2,2,2,2,2
top_pick_flag,False,False,False,False,False
hedging_cost_flag,None,None,False,False,False
owner,superpat,superpat,None,None,None
allow_manual_add,True,True,True,True,True


## Look through

In [27]:
df_facsheet = pd.read_parquet("../app/portprop_data/df_factsheet.parquet")
df_facsheet = df_facsheet.rename(columns={'symbol': 'lookthrough_symbol'})
df_facsheet = df_facsheet.merge(
    df_sec_all[[
        'sec_id', 'symbol',
    ]],
    on='sec_id',
    how='left'
)
df_facsheet

,sec_id,lookthrough_symbol,bm_name,asset_class,weight,symbol
0,145,KKPAGS,SETPREIT50% + FSTREI50%,Alternative,0.000,BBBY:xnas
1,145,KKPAGS,BB Barclay Global Aggregate Hedged-USD (70%) ...,Fixed Income,0.070,BBBY:xnas
2,145,KKPAGS,Bloomberg Commodity TR,Alternative,0.015,BBBY:xnas
3,145,KKPAGS,Dow Jones Brookfield Global Infrastructure Ind...,Alternative,0.045,BBBY:xnas
4,145,KKPAGS,EM Bond Hard Curcy,Fixed Income,0.000,BBBY:xnas
...,...,...,...,...,...,...
1552,104911,SCBWINR,MSCI ACWI,Global Equity,0.350,NaN
1553,104639,KKP SG-AA,BB Barclay Global High Yield Hedged USD,Fixed Income,0.000,NaN
1554,104624,KF-INCOME,US Short Treasury (USD Unhedged),Cash and Cash Equivalent,0.030,NaN
1555,104624,KF-INCOME,BB Barclay Global Aggregate Hedged-USD (70%) ...,Fixed Income,0.000,NaN
